In [4]:
import numpy as np
import pandas as pd
import math
# Visualization
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
from pandas.plotting import scatter_matrix


from sklearn import model_selection, tree, preprocessing, metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

headers = ['Wife_Age', 'Wife_Education',
          'Husband_Education', 'Children_Born',
          'Wife_Religion', 'Wife_Working',
          'Husband_Occupation','SOL_Index',
          'Media_Exposure','Contraceptive_Method_Used']

dataset = pd.read_csv("C:\\Users\\abc86\\Desktop\\(DM)Datamaining\\作業一\\cmc.data",
                   header = None,
                   names = headers,
                   sep = ',')
dataset_raw = dataset.copy()
lowtohigh = { 1: "most_low", 2: "low", 3:"high", 4:"most_high"}
dataset_raw.Wife_Education.replace(lowtohigh, inplace=True)
dataset_raw.Husband_Education.replace(lowtohigh, inplace=True)
dataset_raw.SOL_Index.replace(lowtohigh, inplace=True)
dataset_raw.Husband_Occupation.replace(lowtohigh, inplace=True)

wifereligion = {0:"Non_Islam", 1:"Islam"}
dataset_raw.Wife_Religion.replace(wifereligion, inplace=True)

wifeworking = {0:"Yes", 1:"No"}
dataset_raw.Wife_Working.replace(wifeworking, inplace=True)

mediaexposure = {0: "Good", 1: "Not_Good"}
dataset_raw.Media_Exposure.replace(mediaexposure, inplace=True)

contraceptive = {1: "No_Use", 2:"Long_Term", 3:"Short_Term"}
dataset_raw.Contraceptive_Method_Used.replace(contraceptive, inplace=True)
dataset_raw.head()


,Wife_Age,Wife_Education,Husband_Education,Children_Born,Wife_Religion,Wife_Working,Husband_Occupation,SOL_Index,Media_Exposure,Contraceptive_Method_Used
0,24,low,high,3,Islam,No,low,high,Good,No_Use
1,45,most_low,high,10,Islam,No,high,most_high,Good,No_Use
2,43,low,high,7,Islam,No,high,most_high,Good,No_Use
3,42,high,low,9,Islam,No,high,high,Good,No_Use
4,36,high,high,8,Islam,No,high,low,Good,No_Use


In [5]:
dataset_raw.describe(include=['O'])

,Wife_Education,Husband_Education,Wife_Religion,Wife_Working,Husband_Occupation,SOL_Index,Media_Exposure,Contraceptive_Method_Used
count,1473,1473,1473,1473,1473,1473,1473,1473
unique,4,4,2,2,4,4,2,3
top,most_high,most_high,Islam,No,high,most_high,Good,No_Use
freq,577,899,1253,1104,585,684,1364,629


In [6]:
dataset_raw.dtypes

Wife_Age                      int64
Wife_Education               object
Husband_Education            object
Children_Born                 int64
Wife_Religion                object
Wife_Working                 object
Husband_Occupation           object
SOL_Index                    object
Media_Exposure               object
Contraceptive_Method_Used    object
dtype: object

In [7]:
dataset.head() #our already encoded data

,Wife_Age,Wife_Education,Husband_Education,Children_Born,Wife_Religion,Wife_Working,Husband_Occupation,SOL_Index,Media_Exposure,Contraceptive_Method_Used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1
3,42,3,2,9,1,1,3,3,0,1
4,36,3,3,8,1,1,3,2,0,1


In [9]:
selected_dataset = dataset
selected_dataset.head()

,Wife_Age,Wife_Education,Husband_Education,Children_Born,Wife_Religion,Wife_Working,Husband_Occupation,SOL_Index,Media_Exposure,Contraceptive_Method_Used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1
3,42,3,2,9,1,1,3,3,0,1
4,36,3,3,8,1,1,3,2,0,1


In [10]:
train = selected_dataset.sample(frac=0.7)
test = selected_dataset.loc[~selected_dataset.index.isin(train.index)]
X_train_w_label = train
X_train = train.drop(['Contraceptive_Method_Used'], axis=1)
y_train = train['Contraceptive_Method_Used'].astype('int64')
X_test = test.drop(['Contraceptive_Method_Used'], axis=1)
y_test = test['Contraceptive_Method_Used'].astype('int64')


In [11]:
X_train.head()

,Wife_Age,Wife_Education,Husband_Education,Children_Born,Wife_Religion,Wife_Working,Husband_Occupation,SOL_Index,Media_Exposure
450,23,4,4,1,1,1,3,4,0
1043,45,1,1,0,0,0,2,1,0
229,28,3,4,0,1,1,3,2,0
1465,25,2,4,3,1,1,1,3,0
1159,43,3,4,2,0,1,3,4,0


In [12]:
y_train.head()

450     2
1043    1
229     1
1465    3
1159    1
Name: Contraceptive_Method_Used, dtype: int64

In [13]:
def fit_ml_algo(algo, X_train, y_train, X_test, cv):
    # One pass
    model = algo.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    if (isinstance(algo, ( KNeighborsClassifier,
                         DecisionTreeClassifier,))):
        probs = model.predict_proba(X_test)[:,1]
    else:
        probs = "Not Available"
    acc = round(model.score(X_test, y_test)*100, 2)
    
    # CV
    train_pred = model_selection.cross_val_predict(algo,
                                                  X_train,
                                                  y_train,
                                                  cv=cv,
                                                  n_jobs=-1)
    acc_cv = round(metrics.accuracy_score(y_train, train_pred)*100,2)
    return train_pred, test_pred, acc, acc_cv, probs

In [14]:
from sklearn.neighbors import KNeighborsClassifier
# k-Nearest Neighbors
train_pred_knn, test_pred_knn, acc_knn, acc_cv_knn, probs_knn = fit_ml_algo(
    KNeighborsClassifier(n_neighbors = 3, n_jobs = -1, ), X_train, y_train, X_test, 10)

print("Accuracy: %s" % acc_knn)
print("Accuracy CV 10-Fold: %s" % acc_cv_knn)


Accuracy: 47.51
Accuracy CV 10-Fold: 51.12


In [15]:
print(metrics.classification_report(y_train, train_pred_knn))

              precision    recall  f1-score   support

           1       0.56      0.65      0.60       443
           2       0.47      0.36      0.41       228
           3       0.45      0.44      0.45       360

    accuracy                           0.51      1031
   macro avg       0.50      0.48      0.49      1031
weighted avg       0.51      0.51      0.51      1031



In [16]:
train_pred_dt, test_pred_dt, acc_dt, acc_cv_dt, probs_dt = fit_ml_algo(DecisionTreeClassifier(), 
                                                             X_train, 
                                                             y_train, 
                                                             X_test, 
                                                             10)

print("Accuracy: %s" % acc_dt)
print("Accuracy CV 10-Fold: %s" % acc_cv_dt)


Accuracy: 45.48
Accuracy CV 10-Fold: 48.59


In [17]:
print(metrics.classification_report(y_train, train_pred_dt))

              precision    recall  f1-score   support

           1       0.58      0.59      0.59       443
           2       0.37      0.38      0.37       228
           3       0.44      0.42      0.43       360

    accuracy                           0.49      1031
   macro avg       0.46      0.46      0.46      1031
weighted avg       0.49      0.49      0.49      1031



In [18]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)
model_tree = tree.fit(X_train, y_train)

In [20]:
from itertools import product
from sklearn import tree
import pydotplus




X_featurte =['Wife_Age', 'Wife_Education',
             'Husband_Education', 'Children_Born',
             'Wife_Religion', 'Wife_Working',
             'Husband_Occupation','SOL_Index',
             'Media_Exposure']
y_class =["No_Use","Long_Term","Short_Term"]

dtree = tree.export_graphviz(model_tree, out_file=None,
                             feature_names= X_featurte,
                             class_names=y_class ,
                             filled=True, rounded=True,  
                             special_characters=True
                         )
graph = pydotplus.graph_from_dot_data(dtree)

graph.write_pdf("entr11opy_maxdepth5.pdf")

True